In [3]:
import requests
from bs4 import BeautifulSoup
import json
import time
import pandas as pd

def get_reviews_from_page(url):
    try:
        req = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        req.raise_for_status()  # Raise an error for bad status codes
        time.sleep(2)  # Add a delay to avoid overwhelming the server
        soup = BeautifulSoup(req.text, 'html.parser')
        reviews_raw = soup.find("script", id="__NEXT_DATA__").string
        reviews_raw = json.loads(reviews_raw)
        return reviews_raw["props"]["pageProps"]["reviews"]
    except (requests.RequestException, json.JSONDecodeError, AttributeError) as e:
        return []

def scrape_trustpilot_reviews(base_url: str):
    reviews_data = []

    page_number = 1
    while True:
        url = f"{base_url}?page={page_number}"
        reviews = get_reviews_from_page(url)

        if not reviews:
            break

        for review in reviews:
            data = {
                'Date': pd.to_datetime(review["dates"]["publishedDate"]).strftime("%Y-%m-%d"),
                'Author': review["consumer"]["displayName"],
                'Body': review["text"],
                'Heading': review["title"],
                'Rating': review["rating"],
                'Location': review["consumer"]["countryCode"]
            }
            reviews_data.append(data)
        
        page_number += 1

    # Remove duplicates based on the 'Body' field
    reviews_data = [dict(t) for t in {tuple(d.items()) for d in reviews_data}]
    
    return reviews_data

C:\anaconda3\envs\py39\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.25.11) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [4]:
data = scrape_trustpilot_reviews('https://www.trustpilot.com/review/lazada.sg')

In [7]:
data[0]

{'Date': '2022-11-05',
 'Author': 'Tan Ronald',
 'Body': 'Lazada slow in taking action to scammers.\nMy order $1699 SGD Sterra Tankless water was indicated as delivered.\nDidn’t receive the item. Difficult to contact Lazada, live agent said able to refund. Submit a request But was declined . Lazada just indicated Due to Lazada policy . Saw another buyer post the same review also, about be scammed. Till now the sellers advertisement was not taken down. I Do not trust anymore. Will file to case about this issue.',
 'Heading': 'Lazada slow in action against fraud / scammers',
 'Rating': 1,
 'Location': 'SG'}

In [6]:
df = pd.DataFrame(data)

In [8]:
df.rename(columns={'Body':'Review'},inplace=True)

In [10]:
df.to_excel('trustpilot_reviews.xlsx',index=False)